<a href="https://www.kaggle.com/code/dagartallison/parkinsons-submission-v1?scriptVersionId=128694351" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import sys
sys.path.append('/kaggle/input/amp-pd')

In [2]:
import pandas as pd
import numpy as np
import pickle
import amp_pd_peptide
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [3]:
def preprocess_train_df(train_clin_df, train_prot_df, train_pep_df):
    '''
        Takes in the train_clinical_data.csv, train_peptides.csv, train_proteins.csv as pandas dataframes
        Combines the protein and peptide data names and the joins with the train clinical data
        The dataframes are stratified kfold based on the target
        The function creates one dataframe for each target (updrs_1, updrs_2, updrs_3, updrs_4) stored in the final_df dictionary
        Returns a dictionary of the dataframes for each updrs target
    '''
    
    # drop the medication column
    train_clin_df = train_clin_df.drop(columns=['upd23b_clinical_state_on_medication'])
    
    # create a column with the UniProt and Peptide name combined
    train_pep_df['peptide_uniprot'] = train_pep_df['Peptide'] + '_'+ train_pep_df['UniProt']

    # create a table with the visit_id as the index and the proteins or peptides as the feature and the abundance as the values
    train_prot_pivot = train_prot_df.pivot(index='visit_id', values='NPX', columns='UniProt')
    train_pep_pivot = train_pep_df.pivot(index='visit_id', values='PeptideAbundance', columns='peptide_uniprot')

    # combine the two tables on the visit_id
    full_prot_train_df = train_prot_pivot.join(train_pep_pivot)

    # fill nan with 0 for this first round
    full_prot_train_df = full_prot_train_df.fillna(0)

    full_train_df = train_clin_df.merge(full_prot_train_df, how='inner', left_on='visit_id', right_on='visit_id')
    full_train_df = full_train_df.sample(frac=1).reset_index(drop=True)

    
    updrs = ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']

    final_dfs = dict()

    for target in updrs:
    
        to_remove = [updr for updr in updrs if updr != target]
        
        temp_train_df = full_train_df.drop(to_remove, axis=1)
        temp_train_df = temp_train_df.dropna()
        
        # calculate the number of bins by Sturge's rule
        num_bins = int(np.floor(1 + np.log2(len(full_train_df))))
        temp_train_df.loc[:, "bins"] = pd.cut(temp_train_df[target], bins=num_bins, labels=False)

        temp_train_df = temp_train_df.dropna().reset_index(drop=True)
        
        # initiate the kfold class from sklearn
        kf = StratifiedKFold(n_splits=5)
        
        # create a kfold column
        temp_train_df['kfold'] = -1

        # fill the kfold column
        for f, (t_, v_) in enumerate(kf.split(X=temp_train_df, y=temp_train_df['bins'].values)):
            temp_train_df.loc[v_, 'kfold'] = f
            
        # drop the bins column
        temp_train_df = temp_train_df.drop('bins', axis=1)
        
     

        final_dfs[target] = temp_train_df
            
    return final_dfs

In [4]:
def smape(y_true, y_pred):

    return round(np.mean(np.abs(y_pred - y_true) / ((np.abs(y_true) + np.abs(y_pred))/2)) * 100, 2)



def train_rf_model(df_dict):
    '''
        Takes in the preprocesses training dictionary of dataframes 
        Then trains a random forest regressor model on the data
        Returns a dictionary of models, one for each updrs target
    '''
    model_dict = {}
    visit0_col_dict = {}
    
    updr1_model = RandomForestRegressor(random_state = 42)
    updr2_model = RandomForestRegressor(random_state = 42)
    updr3_model = RandomForestRegressor(random_state = 42)
    updr4_model = RandomForestRegressor(random_state = 42)
    
    for updr in ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']:
        df = df_dict[updr]
        df = df.drop(columns=['visit_id', 'patient_id', 'kfold'])

        y_train = df[updr].values
        df = df.drop(columns=[updr])
        x_train = df.values

        
        if updr == 'updrs_1':
            updr1_model.fit(x_train, y_train)
            preds = updr1_model.predict(x_train)
            r2 = metrics.r2_score(y_train, preds)
            mape = metrics.mean_absolute_percentage_error(y_train, preds)
            s_mape = smape(y_train, preds)
            model_dict[updr] = updr1_model
            visit0_col_dict[updr] = df.columns
            print(f'SMAPE = {s_mape}, R2 = {r2}, MAPE = {mape}')

        elif updr == 'updrs_2':
            updr2_model.fit(x_train, y_train)
            preds = updr2_model.predict(x_train)
            r2 = metrics.r2_score(y_train, preds)
            mape = metrics.mean_absolute_percentage_error(y_train, preds)
            s_mape = smape(y_train, preds)
            model_dict[updr] = updr2_model
            visit0_col_dict[updr] = df.columns
            print(f'SMAPE = {s_mape}, R2 = {r2}, MAPE = {mape}')

        elif updr == 'updrs_3':
            updr3_model.fit(x_train, y_train)
            preds = updr3_model.predict(x_train)
            r2 = metrics.r2_score(y_train, preds)
            mape = metrics.mean_absolute_percentage_error(y_train, preds)
            s_mape = smape(y_train, preds)
            model_dict[updr] = updr3_model
            visit0_col_dict[updr] = df.columns
            print(f'SMAPE = {s_mape}, R2 = {r2}, MAPE = {mape}')
        else:
            updr4_model.fit(x_train, y_train)
            preds = updr4_model.predict(x_train)
            r2 = metrics.r2_score(y_train, preds)
            mape = metrics.mean_absolute_percentage_error(y_train, preds)
            s_mape = smape(y_train, preds)
            model_dict[updr] = updr4_model
            visit0_col_dict[updr] = df.columns
            print(f'SMAPE = {s_mape}, R2 = {r2}, MAPE = {mape}')
        
    
    return model_dict, visit0_col_dict


## Create a Model for Forecasting the future months

In [5]:
def preprocess_forecast_train_df(preprocessed_train_df, target):
    '''
        Takes in the preprocessed training dataframe for a single updrs
        Returns a dataframe for forecasting which has columns for the updrs of different future visits
    '''
    
    temp_df = preprocessed_train_df[['visit_id', 'patient_id', target, 'visit_month']].sort_values(by=['patient_id', 'visit_month']).reset_index(drop=True)
    temp_pivot = temp_df.pivot(columns='visit_month', values=target, index='patient_id')
    temp_pivot = temp_pivot.reset_index()
    
    cols = [f'{target}_{month}' for month in temp_pivot.columns[1:]]
    temp_pivot.columns = ['patient_id'] + cols
    
    forecast_final = preprocessed_train_df[preprocessed_train_df['visit_month'] == 0]
    
    final_df = forecast_final.merge(temp_pivot, on=['patient_id'], how='left')
    
    final_df = final_df.drop(columns=['patient_id', target])
    if 'kfold' in final_df.columns:
        final_df = final_df.drop(columns=['kfold'])
        
    
    return final_df

### Create the forecasting dataframes

In [6]:
def train_forecast(model, processed_forecast_dict, target, month_diff):
    
    # results dictionary for the models
    forecast_model_dict = dict()
    
    # get the training dataset
    df = processed_forecast_dict[target]
    
    forecast_cols = [col for col in df.columns if 'updrs' in col]

    drop_cols = [col for col in forecast_cols if col not in  [f'{target}_0', f'{target}_{month_diff}']]

    df = df.drop(columns=drop_cols)
    df = df.drop(columns=['visit_id', 'visit_month'])
    df = df.rename(columns={f'{target}_0': target})
    
    target_mo = f'{target}_{month_diff}'
    # drop nan rows for target column
    df = df.dropna(subset=[target_mo])
    
    X, y = df, df[target_mo]
    
    X = X.drop([target_mo], axis=1).values
    

    reg = model
    reg.fit(X, y)
    preds = reg.predict(X)
    
    r2 = metrics.r2_score(y, preds)
    mape = metrics.mean_absolute_percentage_error(y, preds)
    s_mape = smape(y, preds)
    
    print(target, month_diff, 'SMAPE:', s_mape)
    
    return model, df.drop(columns=target_mo).columns

In [7]:
def preprocess_test_df(test_clin_df, test_prot_df, test_pep_df):
    '''
        Takes in the test data from the csv file in the form of a pandas dataframe
        Combines the peptide and protein data
        Outputs the dataframe for inference
    '''
    
    
    if 'upd23b_clinical_state_on_medication' in test_clin_df.columns:
        # drop the medication column
        test_clin_df = test_clin_df.drop(columns=['upd23b_clinical_state_on_medication'])
    
    if 'group_key' in test_clin_df.columns:
        # drop the group key
        test_clin_df = test_clin_df.drop(columns=['group_key'])
    
    # create a column with the UniProt and Peptide name combined
    test_pep_df['peptide_uniprot'] = test_pep_df['Peptide'] + '_'+ test_pep_df['UniProt']

    # create a table with the visit_id as the index and the proteins or peptides as the feature and the abundance as the values
    test_prot_pivot = test_prot_df.pivot(index='visit_id', values='NPX', columns='UniProt')
    test_pep_pivot = test_pep_df.pivot(index='visit_id', values='PeptideAbundance', columns='peptide_uniprot')

    # combine the two tables on the visit_id
    full_prot_test_df = test_prot_pivot.join(test_pep_pivot)

    # fill nan with 0 
    full_prot_test_df = full_prot_test_df.fillna(0)

    full_test_df = test_clin_df.merge(full_prot_test_df, how='inner', left_on='visit_id', right_on='visit_id')
    full_test_df = full_test_df.sample(frac=1).reset_index(drop=True)

    missing_row_id = [x for x in test_clin_df['row_id'] if x not in full_test_df['row_id'].to_list()]
    filtered_df = test_clin_df[test_clin_df['row_id'].isin(missing_row_id)]
    imputed_df = filtered_df.drop(columns=['visit_month']).merge(full_test_df.drop(columns=['row_id', 'visit_id']),
                                                                    how='left', 
                                                                    left_on=['patient_id', 'updrs_test'],
                                                                    right_on=['patient_id', 'updrs_test'])
    full_test_df = pd.concat([full_test_df, imputed_df])
    
    full_test_df = full_test_df.reset_index(drop=True)
    
    # remove the imputed visit month and replace from the test df
    full_test_df = full_test_df.drop(columns='visit_month').merge(test_clin_df[['row_id', 'visit_month']], how='left', left_on='row_id', right_on='row_id')
    
    return full_test_df

In [8]:

def prepare_model_df(model_df, target, train_cols, visit_month=0):
    '''
        model_df is the preprocessed test dataframe which has all of the protein data
        target is the updrs number
        train_cols are the list of columns necessary for the model to do the inference
        visit_month is the month of data we want to filter
    '''

    # add visit_month if it is not in the model_df.columns
    if 'visit_month' not in model_df.columns:
        model_df['visit_month'] = visit_month
    
    # start will all the visit_months as 0 for the first prediction
    model_df['visit_month'] = 0
    
    model_df = model_df[model_df['updrs_test'] == target]
    
    # find the columns in preds_cols that are not in the model_df.columns
    not_in_pred_cols = [col for col in train_cols if col not in model_df.columns]

    # create an empty dataframe with the columns in not_in_pred_cols
    not_in_preds_df = pd.DataFrame(columns=not_in_pred_cols)

    # combine the model_df and the not_in_preds_df so all the needed columns are in dataframe
    new_model_df = pd.concat([model_df, not_in_preds_df], axis=1)
    
    # fill the nan values with 0
    new_model_df = new_model_df.fillna(0)
    
    # keep track of the row_id order for later
    row_id_df = new_model_df[['row_id']]

    # filter the new_model_df to only include the columns in pred_cols with the correct order
    return new_model_df[train_cols], row_id_df

## Train the models for Visit 0 and the Forecasting


In [9]:
# read the training data with folds
train_df = pd.read_csv('/kaggle/input/amp-pd/train_clinical_data.csv')
train_prot_df = pd.read_csv('/kaggle/input/amp-pd/train_proteins.csv')
train_pep_df = pd.read_csv('/kaggle/input/amp-pd/train_peptides.csv')

train_df_dict = preprocess_train_df(train_df, train_prot_df, train_pep_df)

In [10]:
train_prot_df[train_prot_df['patient_id'] == 44154]

,visit_id,visit_month,patient_id,UniProt,NPX
27172,44154_0,0,44154,O00391,13303.10
27173,44154_0,0,44154,O00533,396974.00
27174,44154_0,0,44154,O00584,13695.70
27175,44154_0,0,44154,O14498,31938.20
27176,44154_0,0,44154,O14791,5575.59
...,...,...,...,...,...
163811,44154_36,36,44154,Q9UHG2,196829.00
163812,44154_36,36,44154,Q9UKV8,61832.30
163813,44154_36,36,44154,Q9UNU6,18893.60
163814,44154_36,36,44154,Q9Y646,13901.60


In [11]:
df = train_df_dict['updrs_1']
df[df['patient_id'] == 44154]

,visit_id,patient_id,visit_month,updrs_1,O00391,O00533,O00584,O14498,O14773,O14791,...,YTTEIIK_P00736,YVGGQEHFAHLLILR_P02763,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR_P00738,YVMLPVADQDQC(UniMod_4)IR_P00738,YVNKEIQNAVNGVK_P10909,YWGVASFLQK_P02753,YYC(UniMod_4)FQGNQFLR_P02790,YYTYLIMNK_P01024,YYWGGQYTWDMAK_P02675,kfold
489,44154_6,44154,6,0.0,13881.40,465695.0,18877.9,40198.2,10205.5,2596.90,...,12412.2,0.0,32238.1,365352.0,74680.0,89677.9,489700.0,44402.7,31309.7,2
761,44154_0,44154,0,6.0,13303.10,396974.0,13695.7,31938.2,0.0,5575.59,...,11487.7,0.0,63508.4,667561.0,95159.5,97610.4,391721.0,50430.6,27139.7,3
908,44154_36,44154,36,0.0,8360.23,376190.0,13883.4,33677.4,0.0,5034.85,...,10567.2,0.0,56059.4,658461.0,90467.0,99173.1,505380.0,50235.3,39486.1,4


In [12]:
train_df.groupby(['patient_id'])['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4'].max() - train_df.groupby(['patient_id'])['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4'].min()

,updrs_1,updrs_2,updrs_3,updrs_4
patient_id,,,,
55,16.0,15.0,41.0,0.0
942,6.0,7.0,22.0,3.0
1517,12.0,14.0,15.0,7.0
1923,2.0,1.0,3.0,NaN
2660,8.0,0.0,2.0,NaN
...,...,...,...,...
63875,6.0,2.0,4.0,0.0
63889,3.0,8.0,27.0,3.0
64669,11.0,11.0,36.0,0.0


In [13]:
forecast_df = preprocess_forecast_train_df(df, 'updrs_1')
forecast_df[forecast_df['visit_id'] == '44154_0']

,visit_id,visit_month,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,...,updrs_1_24,updrs_1_30,updrs_1_36,updrs_1_48,updrs_1_54,updrs_1_60,updrs_1_72,updrs_1_84,updrs_1_96,updrs_1_108
138,44154_0,0,13303.1,396974.0,13695.7,31938.2,0.0,5575.59,60694.4,54381.9,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
def process_df_for_visit0(train_df, train_df_dict, updr):
    # get the values for visit month 0
    visit_month0_df = train_df[train_df['visit_month'] == 0]
    visit_month0_df = visit_month0_df.rename(columns={'updrs_1': 'updrs_1_visit_0', 'updrs_2': 'updrs_2_visit_0', 'updrs_3': 'updrs_3_visit_0', 'updrs_4': 'updrs_4_visit_0'})
    visit_month0_cols = ['patient_id', f'{updr}_visit_0']
    visit_month0_df = visit_month0_df[visit_month0_cols]
    train_df_updrs_1 = train_df_dict[updr]
    new_train_df = train_df_updrs_1.merge(visit_month0_df, how='left', left_on='patient_id', right_on='patient_id')
    new_train_df = new_train_df.drop(columns=[updr])
    new_train_df = new_train_df.rename(columns={f'{updr}_visit_0':'updrs_1'})
    new_train_df = new_train_df.dropna(subset=[updr])
    return new_train_df

In [15]:
process_df_for_visit0(train_df, train_df_dict, 'updrs_1')

,visit_id,patient_id,visit_month,O00391,O00533,O00584,O14498,O14773,O14791,O15240,...,YVGGQEHFAHLLILR_P02763,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR_P00738,YVMLPVADQDQC(UniMod_4)IR_P00738,YVNKEIQNAVNGVK_P10909,YWGVASFLQK_P02753,YYC(UniMod_4)FQGNQFLR_P02790,YYTYLIMNK_P01024,YYWGGQYTWDMAK_P02675,kfold,updrs_1
0,59550_84,59550,84,9452.83,465506.0,21389.70,19306.8,15675.90,3220.30,147863.0,...,4560150.0,16128.1,204381.0,75196.9,196431.0,571165.0,58959.7,9510.18,0,10.0
1,8699_24,8699,24,9983.00,400290.0,24240.10,0.0,16943.50,6303.17,77493.6,...,8770410.0,33599.1,926094.0,118897.0,133682.0,571879.0,80268.3,54889.70,0,17.0
2,17414_60,17414,60,0.00,762084.0,34485.70,0.0,21803.10,0.00,263634.0,...,2337550.0,48238.3,1335510.0,83119.6,225278.0,646774.0,66494.4,2747.11,0,0.0
3,37566_24,37566,24,0.00,268197.0,18357.80,23754.1,13932.50,2417.44,42532.9,...,4466160.0,78986.6,526682.0,63308.9,117355.0,504129.0,24403.5,12155.60,0,12.0
4,52119_6,52119,6,0.00,347114.0,32629.80,24984.1,13819.10,4278.65,60909.4,...,4189640.0,42269.3,231235.0,93722.2,96308.5,345909.0,25120.2,19278.30,0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063,18183_0,18183,0,12277.40,688535.0,43648.90,25222.4,16022.00,0.00,85958.1,...,4697960.0,68102.1,426377.0,67505.1,149597.0,478390.0,60768.9,12474.10,4,4.0
1064,20216_24,20216,24,15091.00,630350.0,26744.50,34435.3,10782.60,2686.70,127229.0,...,4203150.0,98718.9,1471590.0,165523.0,112659.0,360668.0,49198.8,36288.00,4,1.0
1065,5178_60,5178,60,0.00,539375.0,31362.00,26723.7,17981.70,2060.49,113652.0,...,4458290.0,16453.2,237441.0,96114.2,148504.0,460501.0,52280.6,35382.80,4,9.0
1066,10718_0,10718,0,9563.38,547736.0,18602.80,19873.7,6880.61,1325.22,113248.0,...,0.0,49907.0,864746.0,75000.2,138985.0,655836.0,53866.1,36744.00,4,0.0


In [16]:
    
trained_models_dict, visit0_col_dict = train_rf_model(train_df_dict)
    

processed_forecast_dict = dict()

for updr in ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']:
    
    forecast_data = train_df_dict[updr]
    processed_forecast_dict[updr] = preprocess_forecast_train_df(forecast_data, updr)
    
    
    
    
forecast_col_dict = {'updrs_1':{6:_, 12:_, 24:_}, 'updrs_2':{6:_, 12:_, 24:_}, 'updrs_3':{6:_, 12:_, 24:_}, 'updrs_4':{6:_, 12:_, 24:_}}

model_1_6, model_1_12, model_1_24 = RandomForestRegressor(random_state=42), RandomForestRegressor(random_state=42), RandomForestRegressor(random_state=42)
model_2_6, model_2_12, model_2_24 = RandomForestRegressor(random_state=42), RandomForestRegressor(random_state=42), RandomForestRegressor(random_state=42)
model_3_6, model_3_12, model_3_24 = RandomForestRegressor(random_state=42), RandomForestRegressor(random_state=42), RandomForestRegressor(random_state=42)
model_4_6, model_4_12, model_4_24 = RandomForestRegressor(random_state=42), RandomForestRegressor(random_state=42), RandomForestRegressor(random_state=42)

# store the instantiated models
forecast_dict = {'updrs_1':{6:model_1_6, 12:model_1_12, 24:model_1_24},
                 'updrs_2':{6:model_2_6, 12:model_2_12, 24:model_2_24},
                 'updrs_3':{6:model_3_6, 12:model_3_12, 24:model_3_24},
                 'updrs_4':{6:model_4_6, 12:model_4_12, 24:model_4_24}}


for updr in ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']:
    for month_diff in [6, 12, 24]:
        
        try:
            forecast_model, forecast_cols = train_forecast(forecast_dict[updr][month_diff], processed_forecast_dict, updr, month_diff)
            forecast_dict[updr][month_diff] = forecast_model
            forecast_col_dict[updr][month_diff] = forecast_cols
        except:
            print(f'{updr} {month_diff} forecasting model failed!!!!!')
            
            


SMAPE = 37.16, R2 = 0.8902956666591213, MAPE = 589811310749355.4
SMAPE = 67.67, R2 = 0.8927585718891965, MAPE = 1970830859404633.8
SMAPE = 58.87, R2 = 0.8978839230440281, MAPE = 4021782574608619.0
SMAPE = 129.7, R2 = 0.8668089431428205, MAPE = 1988469831958329.5
updrs_1 6 SMAPE: 37.13
updrs_1 12 SMAPE: 34.61
updrs_1 24 SMAPE: 35.6
updrs_2 6 SMAPE: 22.91
updrs_2 12 SMAPE: 67.1
updrs_2 24 SMAPE: 70.3
updrs_3 6 SMAPE: 20.52
updrs_3 12 SMAPE: 47.78
updrs_3 24 SMAPE: 51.93
updrs_4 6 forecasting model failed!!!!!
updrs_4 12 SMAPE: 127.07
updrs_4 24 SMAPE: 113.64


In [17]:
def fill_test_cols(test_df, pred_cols):
    '''
        Takes in the prediction columns and the test dataframe
        Returns the dataframe with all of the necessary columns for prediction
    '''
    # get the missing columns need for prediction
    missing_cols_from_test = [col for col in pred_cols if col not in test_df.columns]

    # create a dataframe with those columns
    missing_cols_df = pd.DataFrame(columns = missing_cols_from_test)

    # concat these columns to the test_df
    test_df = pd.concat([test_df, missing_cols_df], axis=1)

    # fill the na with 0
    test_df = test_df.fillna(0)

    return test_df[pred_cols]

## Create the Prediction Function

In [18]:
def create_submission(test_df, test_prot_df, test_pep_df, visit0_col_dict, trained_models_dict, forecast_col_dict, forecast_dict):

    '''
    Need to input the following variables:

    test_df = pd.read_csv('/kaggle/input/amp-pd/example_test_files/test.csv')
    test_prot_df = pd.read_csv('/kaggle/input/amp-pd/example_test_files/test_proteins.csv')
    test_pep_df = pd.read_csv('/kaggle/input/amp-pd/example_test_files/test_peptides.csv')
    visit0_col_dict
    trained_models_dict
    forecast_col_dict
    forecast_dict
    '''
    test_preprocessed_df = preprocess_test_df(test_df, test_prot_df, test_pep_df)



    visit0_df = pd.DataFrame()


    final_df = pd.DataFrame()

    # for visit 0
    for updr in ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']:
    
        # predict the 0 visit first
        md_df, row_id = prepare_model_df(test_preprocessed_df, updr, visit0_col_dict[updr], visit_month=0)
        md_df = fill_test_cols(md_df, visit0_col_dict[updr])
        trained_model = trained_models_dict[updr]
        preds = trained_model.predict(md_df.values)
        row_id[f'{updr}'] = preds
        # use value 0 for the updrs_4
        if updr == 'updrs_4':
            row_id['updrs_4'] = 0
        temp_df = pd.concat([row_id, md_df], axis=1)
        visit0_df = pd.concat([visit0_df, temp_df])

        for month in [6, 12, 24]:
            if updr == 'updrs_4' and month == 6:
                # split the difference between 0 and 12
                forecast_cols = forecast_col_dict[updr][12]
                forecast_df = visit0_df.dropna(subset=[updr])
                forecast_id = forecast_df['row_id']
                forecast_df = fill_test_cols(forecast_df, forecast_cols)
                
    
                # get the forecast model
                forecast_model = forecast_dict[updr][12]
                preds = forecast_model.predict(forecast_df.values)
                visit0_preds = forecast_df['updrs_4']
                impute_preds = (preds + visit0_preds) / 2
                # use 0 value for updrs_4 instead of prediction
                forecast_df[f'{updr}_{month}'] = 0
                visit0_df = visit0_df.join(forecast_df[f'{updr}_{month}'])
                
            elif updr == 'updrs_4':
                forecast_cols = forecast_col_dict[updr][month]
                forecast_df = visit0_df.dropna(subset=[updr])
                forecast_id = forecast_df['row_id']
                forecast_df = fill_test_cols(forecast_df, forecast_cols)
    
                # get the forecast model
                forecast_model = forecast_dict[updr][month]
                preds = forecast_model.predict(forecast_df.values)
                # use 0 value for updrs_4 instead of the prediction
                forecast_df[f'{updr}_{month}'] = 0
                visit0_df = visit0_df.join(forecast_df[f'{updr}_{month}'])
            
            else:
                # predict the 6, 12, and 24 later visits
                forecast_cols = forecast_col_dict[updr][month]
                forecast_df = visit0_df.dropna(subset=[updr])
                forecast_id = forecast_df['row_id']
                forecast_df = fill_test_cols(forecast_df, forecast_cols)
    
                # get the forecast model
                forecast_model = forecast_dict[updr][month]
                preds = forecast_model.predict(forecast_df.values)
                forecast_df[f'{updr}_{month}'] = preds
                visit0_df = visit0_df.join(forecast_df[f'{updr}_{month}'])
            
        final_df = pd.concat([final_df, visit0_df])
        final_df = final_df.drop_duplicates()
            
    pred_df = final_df[[col for col in final_df.columns if col == 'row_id' or col[:5] == 'updrs']]

    melted_df = pd.melt(pred_df, id_vars=['row_id'], value_vars=pred_df.columns[1:])
    melted_df = melted_df.dropna()

    for i, row in melted_df.iterrows():
        if row['variable'] in ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']:
            melted_df.loc[i, 'row_id'] = row['row_id'] + '_plus_0_months'
        elif row['variable'] in ['updrs_1_6', 'updrs_2_6', 'updrs_3_6', 'updrs_4_6']:
            melted_df.loc[i, 'row_id'] = row['row_id'] + '_plus_6_months'
        elif row['variable'] in ['updrs_1_12', 'updrs_2_12', 'updrs_3_12', 'updrs_4_12']:
            melted_df.loc[i, 'row_id'] = row['row_id'] + '_plus_12_months'
        else:
            melted_df.loc[i, 'row_id'] = row['row_id'] + '_plus_24_months'
        
    melted_df = melted_df.rename(columns={'value':'rating'}).drop(columns=['variable'])
    result = melted_df.reset_index(drop=True)
    result = result.rename(columns={'row_id':'prediction_id'})
    
    return result

In [19]:
sample = pd.read_csv('/kaggle/input/amp-pd/example_test_files/sample_submission.csv')
sample

,prediction_id,rating,group_key
0,3342_0_updrs_1_plus_0_months,0,0
1,3342_0_updrs_1_plus_6_months,0,0
2,3342_0_updrs_1_plus_12_months,0,0
3,3342_0_updrs_1_plus_24_months,0,0
4,3342_0_updrs_2_plus_0_months,0,0
...,...,...,...
59,50423_6_updrs_3_plus_24_months,0,6
60,50423_6_updrs_4_plus_0_months,0,6
61,50423_6_updrs_4_plus_6_months,0,6
62,50423_6_updrs_4_plus_12_months,0,6


In [20]:
test = pd.read_csv('/kaggle/input/amp-pd/example_test_files/test.csv')
test_proteins = pd.read_csv('/kaggle/input/amp-pd/example_test_files/test_proteins.csv')
test_peptides = pd.read_csv('/kaggle/input/amp-pd/example_test_files/test_peptides.csv')
result = create_submission(test, test_proteins, test_peptides, visit0_col_dict, trained_models_dict, forecast_col_dict, forecast_dict)


In [21]:
pd.set_option('display.max_rows', None)
result.sort_values(by='prediction_id')

,prediction_id,rating
2,3342_0_updrs_1_plus_0_months,5.64
10,3342_0_updrs_1_plus_12_months,5.21
14,3342_0_updrs_1_plus_24_months,5.77
6,3342_0_updrs_1_plus_6_months,6.61
18,3342_0_updrs_2_plus_0_months,3.72
26,3342_0_updrs_2_plus_12_months,5.91
30,3342_0_updrs_2_plus_24_months,5.72
22,3342_0_updrs_2_plus_6_months,6.18
34,3342_0_updrs_3_plus_0_months,19.35
42,3342_0_updrs_3_plus_12_months,18.53


## Submission Through the API

In [22]:
env = amp_pd_peptide.make_env()   # initialize the environment
iter_test = env.iter_test()  

In [23]:
for (test, test_peptides, test_proteins, sample_submission) in iter_test:
        
    result = create_submission(test, test_proteins, test_peptides, visit0_col_dict, trained_models_dict, forecast_col_dict, forecast_dict)

    env.predict(result)   # register your predictions



This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
